In [ ]:
import cv2
import os
import torch
import random

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def extract_and_filter_frames(video_dir, output_dir, minority_class_dir, frames_to_sample=200, interval_seconds=5, frames_per_interval=25):
    # Create separate output directories for each class
    majority_output_dir = os.path.join(output_dir, 'non_shoplifting')
    minority_output_dir = os.path.join(output_dir, 'shoplifting')

    if not os.path.exists(majority_output_dir):
        os.makedirs(majority_output_dir)
    if not os.path.exists(minority_output_dir):
        os.makedirs(minority_output_dir)

    # List all video files
    all_majority_videos = [f for f in os.listdir(video_dir) if f.endswith((".mp4"))]
    all_minority_videos = [f for f in os.listdir(minority_class_dir) if f.endswith((".mp4"))]

    # Randomly sample videos from the minority class for oversampling
    sampled_minority_videos = random.sample(all_minority_videos, min(frames_to_sample, len(all_minority_videos)))

    # Limit the number of frames from the majority class
    max_frames_majority = len(sampled_minority_videos) * frames_per_interval  # Match minority frames

    # Process the majority class but limit the frames
    for video_file in all_majority_videos[:max_frames_majority // frames_per_interval]:
        video_path = os.path.join(video_dir, video_file)
        process_video(video_path, majority_output_dir, interval_seconds, frames_per_interval)

    # Oversample frames from the minority class (shoplifting)
    for video_file in sampled_minority_videos:
        video_path = os.path.join(minority_class_dir, video_file)
        process_video(video_path, minority_output_dir, interval_seconds, frames_per_interval)

def process_video(video_path, output_dir, interval_seconds, frames_per_interval):
    video_capture = cv2.VideoCapture(video_path)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    frames_to_skip = interval_seconds * fps
    count = 0
    success = True

    while success:
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, count * frames_to_skip)

        for i in range(frames_per_interval):
            success, frame = video_capture.read()
            if not success:
                break

            # YOLOv5 person detection
            results = model(frame)

            # Get the labels of detected objects
            labels = results.pred[0][:, -1].cpu().numpy()  # Extract the labels from predictions
            classes = [results.names[int(label)] for label in labels]

            # Only save frames with 'person'
            if 'person' in classes:
                frame_path = os.path.join(output_dir, f"{os.path.basename(video_path)}_frame_{count}_{i}.jpg")
                cv2.imwrite(frame_path, frame)

        count += 1

    video_capture.release()

# Example usage
video_dir = '/kaggle/input/shop-dataset/Shop DataSet/non shop lifters'  # Majority class (non-shoplifters)
minority_class_dir = '/kaggle/input/shop-dataset/Shop DataSet/shop lifters'  # Minority class (shoplifters)
output_dir = 'filtered_frames_output_dir'

extract_and_filter_frames(video_dir, output_dir, minority_class_dir, frames_to_sample=200, interval_seconds=4, frames_per_interval=6)


In [14]:
import os

# Set the directory path
shoplifting_dir = '/kaggle/working/filtered_frames_output_dir/non_shoplifting'

# Get the number of files in the directory
num_files = len([f for f in os.listdir(shoplifting_dir) if os.path.isfile(os.path.join(shoplifting_dir, f))])

# Print the result
print(f"Number of files in shoplifting directory: {num_files}")

Number of files in shoplifting directory: 2806


In [15]:
import os

# Set the directory path
shoplifting_dir = '/kaggle/working/filtered_frames_output_dir/shoplifting'

# Get the number of files in the directory
num_files = len([f for f in os.listdir(shoplifting_dir) if os.path.isfile(os.path.join(shoplifting_dir, f))])

# Print the result
print(f"Number of files in shoplifting directory: {num_files}")

Number of files in shoplifting directory: 2918


In [ ]:
!zip -r file.zip /kaggle/working/filtered_frames_output_dir


In [ ]:
import os
import cv2
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def label_frames(frames_dir, labels_dir):
    """
    Function to label frames with YOLO format annotations and save them into a separate directory.

    Parameters:
    - frames_dir (str): Path to the directory containing frames.
    - labels_dir (str): Path to the directory to save corresponding YOLO annotation files.
    """

    # Ensure the labels directory exists
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)

    # Loop through each frame in the frames directory
    for frame_filename in os.listdir(frames_dir):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frames_dir, frame_filename)

            # Load frame
            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"Error loading {frame_filename}, skipping.")
                continue

            # YOLOv5 person detection
            results = model(frame)

            # Get the labels and bounding boxes of detected objects
            labels = results.pred[0][:, -1].cpu().numpy()  # Class labels
            boxes = results.pred[0][:, :4].cpu().numpy()  # Bounding boxes
            classes = [results.names[int(label)] for label in labels]

            # Only save annotations if 'person' is detected
            if 'person' in classes:
                # Create corresponding YOLO annotation file
                txt_filename = frame_filename.replace('.jpg', '.txt').replace('.png', '.txt')
                txt_path = os.path.join(labels_dir, txt_filename)

                with open(txt_path, 'w') as f:
                    for i, cls in enumerate(classes):
                        if cls == 'person':
                            # YOLO format: <class> <x_center> <y_center> <width> <height>
                            box = boxes[i]
                            x_center = (box[0] + box[2]) / 2 / frame.shape[1]
                            y_center = (box[1] + box[3]) / 2 / frame.shape[0]
                            width = (box[2] - box[0]) / frame.shape[1]
                            height = (box[3] - box[1]) / frame.shape[0]

                            # Write annotation in YOLO format (class 0 for person)
                            f.write(f"0 {x_center} {y_center} {width} {height}\n")

                print(f"Saved annotation for {frame_filename}")
            else:
                print(f"No person detected in {frame_filename}")

# Example usage
frames_dir = '/kaggle/working/filtered_frames_output_dir/non_shoplifting'  
labels_dir = '/kaggle/working/non_shoplifting_labels'  

label_frames(frames_dir, labels_dir)


In [ ]:
import os
import cv2
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def label_frames(frames_dir, labels_dir):
    """
    Function to label frames with YOLO format annotations and save them into a separate directory.

    Parameters:
    - frames_dir (str): Path to the directory containing frames.
    - labels_dir (str): Path to the directory to save corresponding YOLO annotation files.
    """

    # Ensure the labels directory exists
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)

    # Loop through each frame in the frames directory
    for frame_filename in os.listdir(frames_dir):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frames_dir, frame_filename)

            # Load frame
            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"Error loading {frame_filename}, skipping.")
                continue

            # YOLOv5 person detection
            results = model(frame)

            # Get the labels and bounding boxes of detected objects
            labels = results.pred[0][:, -1].cpu().numpy()  # Class labels
            boxes = results.pred[0][:, :4].cpu().numpy()  # Bounding boxes
            classes = [results.names[int(label)] for label in labels]

            # Only save annotations if 'person' is detected
            if 'person' in classes:
                # Create corresponding YOLO annotation file
                txt_filename = frame_filename.replace('.jpg', '.txt').replace('.png', '.txt')
                txt_path = os.path.join(labels_dir, txt_filename)

                with open(txt_path, 'w') as f:
                    for i, cls in enumerate(classes):
                        if cls == 'person':
                            # YOLO format: <class> <x_center> <y_center> <width> <height>
                            box = boxes[i]
                            x_center = (box[0] + box[2]) / 2 / frame.shape[1]
                            y_center = (box[1] + box[3]) / 2 / frame.shape[0]
                            width = (box[2] - box[0]) / frame.shape[1]
                            height = (box[3] - box[1]) / frame.shape[0]

                            # Write annotation in YOLO format (class 0 for person)
                            f.write(f"0 {x_center} {y_center} {width} {height}\n")

                print(f"Saved annotation for {frame_filename}")
            else:
                print(f"No person detected in {frame_filename}")

# Example usage
frames_dir = '/kaggle/working/filtered_frames_output_dir/shoplifting'  
labels_dir = '/kaggle/working/shoplifting_labels'  

label_frames(frames_dir, labels_dir)


In [ ]:
import os
import cv2

def visualize_annotations(frames_dir, labels_dir, output_dir=None):
    """
    Visualize YOLO annotations by drawing bounding boxes on the images.

    Parameters:
    - frames_dir (str): Path to the directory containing frames.
    - labels_dir (str): Path to the directory containing YOLO annotation files.
    - output_dir (str): Optional path to save the annotated images. If not provided, images will be shown using OpenCV's imshow.
    """

    # Ensure the output directory exists if provided
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through each frame in the frames directory
    for frame_filename in os.listdir(frames_dir):
        if frame_filename.endswith(('.jpg', '.png', '.jpeg')):
            frame_path = os.path.join(frames_dir, frame_filename)
            label_path = os.path.join(labels_dir, frame_filename.replace('.jpg', '.txt').replace('.png', '.txt'))

            # Load the image
            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"Error loading {frame_filename}, skipping.")
                continue

            # Check if annotation file exists
            if not os.path.exists(label_path):
                print(f"No annotation for {frame_filename}, skipping.")
                continue

            # Read the annotation file
            with open(label_path, 'r') as f:
                lines = f.readlines()

            # Loop through each line in the annotation file
            for line in lines:
                # YOLO format: <class_id> <x_center> <y_center> <width> <height>
                class_id, x_center, y_center, width, height = map(float, line.strip().split())

                # Convert normalized YOLO coordinates back to image coordinates
                img_height, img_width, _ = frame.shape
                x_center *= img_width
                y_center *= img_height
                width *= img_width
                height *= img_height

                # Calculate the top-left and bottom-right corners of the bounding box
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)

                # Draw the bounding box on the image (use green for 'person')
                color = (0, 255, 0)  # Green for 'person'
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Save or display the annotated image
            if output_dir:
                output_path = os.path.join(output_dir, frame_filename)
                cv2.imwrite(output_path, frame)
                print(f"Annotated image saved at {output_path}")
            else:
                # Display the image
                cv2.imshow('Annotated Image', frame)
                cv2.waitKey(0)  # Wait for a key press to close the image window
                cv2.destroyAllWindows()

# Example usage
frames_dir = '/kaggle/working/filtered_frames_output_dir/shoplifting' 
labels_dir = '/kaggle/working/shoplifting_labels'  
output_dir = '/kaggle/working/shoplifting_labelss'  

visualize_annotations(frames_dir, labels_dir, output_dir)


In [ ]:
!zip -r label_file.zip /kaggle/working/non_shoplifting_labels


In [26]:
!zip -r shoplifting_label_file.zip /kaggle/working/shoplifting_labels


  adding: kaggle/working/shoplifting_labels/ (stored 0%)
  adding: kaggle/working/shoplifting_labels/videyyyyyyzzzzzyyyss_17.mp4_frame_1_4.txt (deflated 29%)
  adding: kaggle/working/shoplifting_labels/shop_lifter_88.mp4_frame_1_1.txt (deflated 28%)
  adding: kaggle/working/shoplifting_labels/videyyyyyynnnnnnzzzzzyyyss_11.mp4_frame_0_5.txt (deflated 27%)
  adding: kaggle/working/shoplifting_labels/videmmmmmmsss_93.mp4_frame_2_5.txt (deflated 29%)
  adding: kaggle/working/shoplifting_labels/shop_lifter_67.mp4_frame_0_5.txt (deflated 27%)
  adding: kaggle/working/shoplifting_labels/shop_lifter_68.mp4_frame_1_2.txt (deflated 27%)
  adding: kaggle/working/shoplifting_labels/videmmmmmmsss_88.mp4_frame_1_0.txt (deflated 26%)
  adding: kaggle/working/shoplifting_labels/videyyyyyynnnnnnzzzzzyyyss_5.mp4_frame_0_5.txt (deflated 30%)
  adding: kaggle/working/shoplifting_labels/videppppsss_18.mp4_frame_2_5.txt (deflated 28%)
  adding: kaggle/working/shoplifting_labels/videyyyyyyzzzzzyyyss_8.mp4_fr